# Load Data

In [ ]:
import matplotlib.pyplot as plt
import xarray as xr
import util

plt.rcParams['figure.figsize'] = [16, 3]

import sys
sys.path.append('/mnt/lustre/IAM851/jm1667/psc/python')
import psc

import logging
# logging.basicConfig(level=logging.DEBUG)

%config InlineBackend.figure_format = 'retina'

In [2]:
case = "case1"
B = 1
res = 512
mod = ["", "_v-1", '_v0', '_dup', '_T0', '_v2'][0]

run = util.Run(case, B, res, mod)
print(run.path)

/mnt/lustre/IAM851/jm1667/psc-runs/case1/trials/B1_n512/


In [3]:
run.initFMNS(nframes=10)
run.printMetadata()

nsteps in sim: 1000000
nframes in animation = 10
steps per frame: 100000
directory to save in: case1/B1_n512_10x100000


In [4]:
import matplotlib.animation as animation
from IPython.display import HTML
import numpy as np

In [5]:
paramsId = 0
params = [
        util.DataParams('Electron Density', 0, None, 'inferno', 'pfd_moments', 'rho_e', coef=-1),
        util.DataParams('Ion Density', 0, None, 'inferno', 'pfd_moments', 'rho_i'),
        util.DataParams('Y-ial E', -.0005, .0005, 'RdBu', 'pfd', 'ey_ec'),
        util.DataParams('Y-ial B', -3e-8, 3e-8, 'RdBu', 'pfd', 'hy_fc', skipFirst=True),
        util.DataParams('X-ial B', -B*1.2, B*1.2, 'RdBu', 'pfd', 'hx_fc'),
        util.DataParams('Y-ial J', -.0005, .0005, 'RdBu', 'pfd', 'jy_ec', skipFirst=True),
        util.DataParams('Radial J', None, None, 'RdBu', 'pfd', ['jy_ec', 'jz_ec'], skipFirst=True, combine='radial'),
        util.DataParams('Azimuthal J', None, None, 'RdBu', 'pfd', ['jy_ec', 'jz_ec'], skipFirst=True, combine='azimuthal'),
        util.DataParams('Electron Temperature', 0, None, 'inferno', 'pfd_moments', ['tyy_e', 'tzz_e'], skipFirst=False, combine='sum'),
        ][paramsId]
print(f"quantity: {params.title}")

quantity: Electron Density


In [6]:
# load data
run.loadData(params)

In [7]:
# slice data
run.sliceData(sliceID = 1)
run.printViewInfo()

view: Central = slice(-0.003, 0.003, None)


In [8]:
# view a particular frame
%matplotlib ipympl

run.viewFrame(frameIdx=0)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Stability

In [9]:
%matplotlib ipympl

run.viewStability()
run.printLmis()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

local min indices: [0, 4, 9]


# Radial Dependence

In [10]:
run.prepRadial()

In [12]:
minMeanses, minTimes = run.getMeansVsRadiusesAndTimes(run.lmis)
maxMeanses, maxTimes = run.getMeansVsRadiusesAndTimes((lmi1 + lmi2) // 2 for lmi1, lmi2 in zip(run.lmis[:-1], run.lmis[1:]))
midMeanses, midTimes = run.getMeansVsRadiusesAndTimes((3*lmi1 + lmi2) // 4 for lmi1, lmi2 in zip(run.lmis[:-1], run.lmis[1:]))
means0 = run.getMeansVsRadius(0)
# minmeanses, minstdses = tuple(zip(*[getMeansAndStds(slicedDatas[lmi]) for lmi in lmis]))
# maxmeanses, maxstdses = tuple(zip(*[getMeansAndStds(slicedDatas[(lmi1 + lmi2) // 2]) for lmi1, lmi2 in zip(lmis[:-1], lmis[1:])]))
# midmeanses, midstdses = tuple(zip(*[getMeansAndStds(slicedDatas[(3 * lmi1 + lmi2) // 4]) for lmi1, lmi2 in zip(lmis[:-1], lmis[1:])]))
# means0, stds0 = getMeansAndStds(slicedDatas[0])

In [14]:
# mean values over a specific number of periods
lmii = 0
npers = 2
permeanses, pertimes = run.getMeansVsRadiusesAndTimes(range(run.lmis[lmii], run.lmis[lmii+npers]))
# permeanses, perstdses = tuple(zip(*[getMeansAndStds(slicedDatas[lmi]) for lmi in range(lmis[lmii], lmis[lmii+npers])]))

In [15]:
# time average of mean values from a period
timeav = np.array(permeanses).mean(axis=0)

In [24]:
%matplotlib ipympl
from matplotlib import cm



##### plot sequence of lmins or whatever
plt.plot(run.rs, means0, color="black", label="t={:.2f}".format(0))
for i in range(len(minMeanses)):
    plt.plot(run.rs, minMeanses[i], color=cm.Wistia(1-i/len(minMeanses)), label="t={:.2f}".format(run.times[run.lmis[i]]))
# for i in range(len(maxmeanses)):
#     plt.plot(rs, maxmeanses[i], color=cm.Wistia(1-i/len(maxmeanses)), label="t={:.2f}".format(times[(lmis[i] + lmis[i+1])//2]))
# for i in range(len(midmeanses)):
#     plt.plot(rs, midmeanses[i], color=cm.Wistia(1-i/len(midmeanses)), label="t={:.2f}".format(times[(3 * lmis[i] + lmis[i+1])//4]))

##### plot time average over a period
# plt.plot(rs, timeav, label="t:[{:.2f},{:.2f}]".format(times[lmis[lmii]], times[lmis[lmii+npers]]))

# plt.legend()
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [42]:
##### plot times over a period
fig, ax = plt.subplots(subplot_kw={"projection": "3d"})

X, Y = np.meshgrid(rs, times[lmis[lmii]:lmis[lmii+npers]])
# Z = np.array(permeanses)
surf = ax.plot_surface(X, Y, Z, cmap=cm.coolwarm, linewidth=0, antialiased=False)

Z = np.array([subtract(permean, timeav) for permean in permeanses])
surf = ax.plot_surface(X, Y, Z, cmap=cm.coolwarm, vmin=-.15, vmax=.15, linewidth=0, antialiased=False)

ax.set_xlim(0, .003)
fig.colorbar(surf, shrink=0.5, aspect=5)

plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …